# Setup

In [ ]:
# Se adjunta esta libreria porque ultralytics puede dar errores
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Importamos librerias
import shutil
from PIL import Image, ImageDraw
from matplotlib.pyplot import imshow
import numpy as np
import glob
import os
import re
from datetime import datetime
from pathlib import Path

In [ ]:
# Instalamos la librería de ultralytics que contiene YOLOv11
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 18.9 MB/s eta 0:00:00


In [ ]:
# Creamos directorio de datasets y importamos ficheros de configuracion
!mkdir datasets
!cp drive/MyDrive/UOC/DELT/DELTv11.yaml .
!cp drive/MyDrive/UOC/DELT/.comet.config .

In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Esta función nos dará la lista de ficheros a mover desde Drive
def full_name_func(source_folder,dataset,file_exclusion_patterns = [],subdirectories_included = [],frames = 1):

    full_name_list = []

    for root, dirs, files in os.walk(source_folder):
        for directory in dirs:
            if directory == 'images':
                source_images_path = os.path.join(root, directory)
                for file in os.listdir(source_images_path):
                    if not file.startswith(tuple(file_exclusion_patterns)):
                        src_file = os.path.join(source_images_path, file)
                        subfolder = os.path.basename(Path(src_file).parents[1])
                        if ((subfolder in subdirectories_included) or (not subdirectories_included)):
                            full_name = dataset + '-' + subfolder +'-'+ file
                            full_name_list.append(full_name[:-4])

    full_name_list.sort()

    final_list = [file for i,file in enumerate(full_name_list) if i%frames == 0]

    return final_list

In [ ]:
# Movemos la lista de ficheros de Drive a Colab
def drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list):

    os.makedirs(destination_images, exist_ok=True)
    os.makedirs(destination_labels, exist_ok=True)

    for root, dirs, files in os.walk(source_folder):
        for directory in dirs:
            if directory == 'images':
                source_images_path = os.path.join(root, directory)
                for file in os.listdir(source_images_path):
                    src_file = os.path.join(source_images_path, file)
                    dst_file = os.path.join(destination_images, file)

                    subfolder = os.path.basename(Path(src_file).parents[1])
                    full_name = dataset + '-' + subfolder +'-'+ file
                    new_path = os.path.join(destination_images, full_name)

                    if full_name[:-4] in file_list:
                        shutil.copy(src_file, dst_file)
                        shutil.move(dst_file, new_path)
            elif directory == 'labels':
                source_labels_path = os.path.join(root, directory)
                for file in os.listdir(source_labels_path):
                    src_file = os.path.join(source_labels_path, file)
                    dst_file = os.path.join(destination_labels, file)

                    subfolder = os.path.basename(Path(src_file).parents[1])
                    full_name = dataset + '-' + subfolder +'-'+ file
                    new_path = os.path.join(destination_labels, full_name)

                    if full_name[:-4] in file_list:
                        shutil.copy(src_file, dst_file)
                        shutil.move(dst_file, new_path)

In [ ]:
destination_images = '/content/datasets/DELT/images'
destination_labels = '/content/datasets/DELT/labels'

dataset = 'osaconservation'
n_frames = 1

source_folder = '/content/drive/MyDrive/UOC/DELT/' + dataset

# Primer test:
# s_included = ['PreArribada','102FTASK','103FTASK','105FTASK','111FTASK','116FTASK','118FTASK','121FTASK','124FTASK']
# Segundo test + test YouTube:
s_included = ['PreArribada','106FTASK','108FTASK','113FTASK','114FTASK','119FTASK','120FTASK','122FTASK','125FTASK']

file_list = full_name_func(source_folder,dataset,subdirectories_included = s_included,frames = n_frames)
drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list)

dataset = 'youtubenew'
n_frames = 1

s_included = []

source_folder = '/content/drive/MyDrive/UOC/DELT/' + dataset
file_list = full_name_func(source_folder,dataset,subdirectories_included = s_included,frames=n_frames)
drive_to_colab(source_folder,destination_images,destination_labels,dataset,file_list)

In [ ]:
# Creamos la estructura de train/val/test
if not os.path.exists('DELT'):
      for split in ['train','val', 'test']:
          os.makedirs(f'datasets/DELT/{split}/images')
          os.makedirs(f'datasets/DELT/{split}/labels')

In [ ]:
# Spliteamos las imagenes y labels

def get_filenames(folder):
    filenames = set()

    for path in glob.glob(os.path.join(folder, '*.*')):

        filename = os.path.split(path)[-1]
        filenames.add(filename)

    return filenames

def split_dataset(image_names, train_size, val_size):
    for i, image_name in enumerate(image_names):

        label_name = image_name.replace('.JPG', '.txt')
        label_name = label_name.replace('.jpg', '.txt')

        if i < train_size:
            split = 'train'
        elif i < train_size + val_size:
            split = 'val'
        else:
            split = 'test'

        source_image_path = f'datasets/DELT/images/{image_name}'
        source_label_path = f'datasets/DELT/labels/{label_name}'

        target_image_folder = f'datasets/DELT/{split}/images'
        target_label_folder = f'datasets/DELT/{split}/labels'

        shutil.copy(source_image_path, target_image_folder)
        try:
          shutil.copy(source_label_path, target_label_folder) # No todas las imagenes tienen label
        except FileNotFoundError:
          pass

turtle_images = get_filenames('datasets/DELT/images')
turtle_images = np.array(list(turtle_images))

np.random.seed(42)
np.random.shuffle(turtle_images)

train_size = int(np.floor(len(turtle_images)*0.8))
val_size = int(np.floor(len(turtle_images)*0.2))

split_dataset(turtle_images, train_size, val_size)

In [ ]:
!pip install comet_ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.9/709.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.8/981.8 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.2.0
    Uninstalling python-box-7.2.0:
      Successfully uninstalled python-box-7.2.0


In [ ]:
# Usamos Comet ML
import comet_ml

comet_ml.init()

COMET WARNING: comet_ml.init() is deprecated and will be removed soon. Please use comet_ml.login()


# Entrenamiento

In [ ]:
#@title Select YOLOv11 🚀 model {run: 'auto'}
YOLO_model = 'yolo11n' #@param ['yolo11n','yolo11s', 'yolo11m', 'yolo11l','yolo11x']

In [ ]:
dataset_name = 'osaconservation' + '-youtube'

# Cargamos el modelo
model = YOLO(YOLO_model)

date_string = datetime.now().strftime("%Y%m%d%H%M")
project_name = dataset_name + '-' + date_string + '-' + YOLO_model
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

results = model.train(data="DELTv11.yaml", epochs=300, freeze=11,patience = 15, project = project_folder, name = project_name)

100%|██████████| 5.35M/5.35M [00:00<00:00, 128MB/s]


Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=DELTv11.yaml, epochs=300, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-youtube-202412061432-yolo11n, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=11, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sav

100%|██████████| 755k/755k [00:00<00:00, 25.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/victorcancer/content-drive-mydrive-uoc-delt-weights/ff9ca166246a43b7936b53ea45806ceb

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


...

                   all        822       1665      0.917      0.852      0.927      0.628
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 94, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



109 epochs completed in 2.772 hours.
Optimizer stripped from /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412061432-yolo11n/weights/last.pt, 5.5MB
Optimizer stripped from /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412061432-yolo11n/weights/best.pt, 5.5MB

Validating /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412061432-yolo11n/weights/best.pt...
Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:11<00:00,  2.30it/s]


                   all        822       1665      0.918      0.841      0.925      0.632
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412061432-yolo11n


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : blush_nation_6426
COMET INFO:     url                   : https://www.comet.com/victorcancer/content-drive-mydrive-uoc-delt-weights/ff9ca166246a43b7936b53ea45806ceb
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [219]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     lr/pg1 [219]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     lr/pg2 [219]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     metrics/mAP50(B) [220]     : (0.75215, 0.92853)
COMET INFO:     metrics/mAP50-95(B) [220]  : (0.41384, 0.63266)
COMET INFO:     metrics/precision(B) [220] : (0.74363, 0

In [ ]:
#@title Select YOLOv11 🚀 model {run: 'auto'}
YOLO_model = 'yolo11s' #@param ['yolo11n','yolo11s', 'yolo11m', 'yolo11l','yolo11x']

In [ ]:
dataset_name = 'osaconservation' + '-youtube'

# Cargamos el modelo
model = YOLO(YOLO_model)

date_string = datetime.now().strftime("%Y%m%d%H%M")
project_name = dataset_name + '-' + date_string + '-' + YOLO_model
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# Train the model
results = model.train(data="DELTv11.yaml", epochs=300, freeze=11,patience = 15, project = project_folder, name = project_name)

100%|██████████| 18.4M/18.4M [00:00<00:00, 135MB/s] 


Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=DELTv11.yaml, epochs=300, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-youtube-202412061719-yolo11s, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=11, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sav

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/victorcancer/content-drive-mydrive-uoc-delt-weights/70b96d4df8c148e4bc33642b25b37ebf

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


TensorBoard: Start with 'tensorboard --logdir /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412061719-yolo11s', view at http://localhost:6006/
Freezing layer 'model.0.conv.weight'
Freezing layer 'model.0.bn.weight'
Freezing layer 'model.0.bn.bias'
Freezing layer 'model.1.conv.weight'
Freezing layer 'model.1.bn.weight'
Freezing layer 'model.1.bn.bias'
Freezing layer 'model.2.cv1.conv.weight'
Freezing layer 'model.2.cv1.bn.weight'
Freezing layer 'model.2.cv1.bn.bias'
Freezing layer 'model.2.cv2.conv.weight'
Freezing layer 'model.2.cv2.bn.weight'
Freezing layer 'model.2.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv1.conv.weight'
Freezing layer 'model.2.m.0.cv1.bn.weight'
Freezing layer 'model.2.m.0.cv1.bn.bias'
Freezing layer 'model.2.m.0.cv2.conv.weight'
Freezing layer 'model.2.m.0.cv2.bn.weight'
Freezing layer 'model.2.m.0.cv2.bn.bias'
Freezing layer 'model.3.conv.weight'
Freezing layer 'model.3.bn.weight'
Freezing layer 'model.3.bn.bias'
Freezing layer 'model.4.cv1.c

train: Scanning /content/datasets/DELT/train/labels.cache... 2601 images, 689 backgrounds, 2 corrupt: 100%|██████████| 3290/3290 [00:00<?, ?it/s]

...

                   all        822       1665      0.925      0.886      0.942      0.656
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 134, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



149 epochs completed in 4.086 hours.
Optimizer stripped from /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412061719-yolo11s/weights/last.pt, 19.2MB
Optimizer stripped from /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412061719-yolo11s/weights/best.pt, 19.2MB

Validating /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412061719-yolo11s/weights/best.pt...
Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11s summary (fused): 238 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:13<00:00,  1.91it/s]


                   all        822       1665      0.942      0.873      0.944      0.657
Speed: 0.2ms preprocess, 3.5ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412061719-yolo11s


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : dual_monument_5746
COMET INFO:     url                   : https://www.comet.com/victorcancer/content-drive-mydrive-uoc-delt-weights/70b96d4df8c148e4bc33642b25b37ebf
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [299]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     lr/pg1 [299]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     lr/pg2 [299]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     metrics/mAP50(B) [300]     : (0.78876, 0.9448)
COMET INFO:     metrics/mAP50-95(B) [300]  : (0.4461, 0.6572)
COMET INFO:     metrics/precision(B) [300] : (0.7713, 0.95

In [ ]:
#@title Select YOLOv11 🚀 model {run: 'auto'}
YOLO_model = 'yolo11m' #@param ['yolo11n','yolo11s', 'yolo11m', 'yolo11l','yolo11x']

In [ ]:
dataset_name = 'osaconservation' + '-youtube'

# Cargamos el modelo
model = YOLO(YOLO_model)

date_string = datetime.now().strftime("%Y%m%d%H%M")
project_name = dataset_name + '-' + date_string + '-' + YOLO_model
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# Train the model
results = model.train(data="DELTv11.yaml", epochs=300, freeze=11,patience = 15, project = project_folder, name = project_name)

100%|██████████| 38.8M/38.8M [00:00<00:00, 130MB/s]


Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=DELTv11.yaml, epochs=300, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-youtube-202412062125-yolo11m, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=11, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sav

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/victorcancer/content-drive-mydrive-uoc-delt-weights/6269071431784a778fc9b86369e1fc3d

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


TensorBoard: Start with 'tensorboard --logdir /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412062125-yolo11m', view at http://localhost:6006/
Freezing layer 'model.0.conv.weight'
Freezing layer 'model.0.bn.weight'
Freezing layer 'model.0.bn.bias'
Freezing layer 'model.1.conv.weight'
Freezing layer 'model.1.bn.weight'
Freezing layer 'model.1.bn.bias'
Freezing layer 'model.2.cv1.conv.weight'
Freezing layer 'model.2.cv1.bn.weight'
Freezing layer 'model.2.cv1.bn.bias'
Freezing layer 'model.2.cv2.conv.weight'
Freezing layer 'model.2.cv2.bn.weight'
Freezing layer 'model.2.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv1.conv.weight'
Freezing layer 'model.2.m.0.cv1.bn.weight'
Freezing layer 'model.2.m.0.cv1.bn.bias'
Freezing layer 'model.2.m.0.cv2.conv.weight'
Freezing layer 'model.2.m.0.cv2.bn.weight'
Freezing layer 'model.2.m.0.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv3.conv.weight'
Freezing layer 'model.2.m.0.cv3.bn.weight'
Freezing layer 'model.2.m.0.cv3.bn.bias'
Freez

train: Scanning /content/datasets/DELT/train/labels.cache... 2601 images, 689 backgrounds, 2 corrupt: 100%|██████████| 3290/3290 [00:00<?, ?it/s]

...

                   all        822       1665      0.947       0.85      0.943      0.654
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 70, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



85 epochs completed in 2.660 hours.
Optimizer stripped from /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412062125-yolo11m/weights/last.pt, 40.5MB
Optimizer stripped from /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412062125-yolo11m/weights/best.pt, 40.5MB

Validating /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412062125-yolo11m/weights/best.pt...
Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11m summary (fused): 303 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:15<00:00,  1.68it/s]


                   all        822       1665      0.931      0.868      0.945      0.654
Speed: 0.2ms preprocess, 7.9ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412062125-yolo11m


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : envious_cottage_8331
COMET INFO:     url                   : https://www.comet.com/victorcancer/content-drive-mydrive-uoc-delt-weights/6269071431784a778fc9b86369e1fc3d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [171]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     lr/pg1 [171]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     lr/pg2 [171]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     metrics/mAP50(B) [172]     : (0.82463, 0.94824)
COMET INFO:     metrics/mAP50-95(B) [172]  : (0.47971, 0.65434)
COMET INFO:     metrics/precision(B) [172] : (0.80392

In [ ]:
#@title Select YOLOv11 🚀 model {run: 'auto'}
YOLO_model = 'yolo11l' #@param ['yolo11n','yolo11s', 'yolo11m', 'yolo11l','yolo11x']

In [ ]:
dataset_name = 'osaconservation' + '-youtube'

# Cargamos el modelo
model = YOLO(YOLO_model)

date_string = datetime.now().strftime("%Y%m%d%H%M")
project_name = dataset_name + '-' + date_string + '-' + YOLO_model
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# Train the model
results = model.train(data="DELTv11.yaml", epochs=300, freeze=11,patience = 15, project = project_folder, name = project_name)

100%|██████████| 49.0M/49.0M [00:00<00:00, 118MB/s]


Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=DELTv11.yaml, epochs=300, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-youtube-202412070005-yolo11l, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=11, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sav

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/victorcancer/content-drive-mydrive-uoc-delt-weights/bf8bf02aa32d4b23a18fa52322cc0647

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


TensorBoard: Start with 'tensorboard --logdir /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412070005-yolo11l', view at http://localhost:6006/
Freezing layer 'model.0.conv.weight'
Freezing layer 'model.0.bn.weight'
Freezing layer 'model.0.bn.bias'
Freezing layer 'model.1.conv.weight'
Freezing layer 'model.1.bn.weight'
Freezing layer 'model.1.bn.bias'
Freezing layer 'model.2.cv1.conv.weight'
Freezing layer 'model.2.cv1.bn.weight'
Freezing layer 'model.2.cv1.bn.bias'
Freezing layer 'model.2.cv2.conv.weight'
Freezing layer 'model.2.cv2.bn.weight'
Freezing layer 'model.2.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv1.conv.weight'
Freezing layer 'model.2.m.0.cv1.bn.weight'
Freezing layer 'model.2.m.0.cv1.bn.bias'
Freezing layer 'model.2.m.0.cv2.conv.weight'
Freezing layer 'model.2.m.0.cv2.bn.weight'
Freezing layer 'model.2.m.0.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv3.conv.weight'
Freezing layer 'model.2.m.0.cv3.bn.weight'
Freezing layer 'model.2.m.0.cv3.bn.bias'
Freez

train: Scanning /content/datasets/DELT/train/labels.cache... 2601 images, 689 backgrounds, 2 corrupt: 100%|██████████| 3290/3290 [00:00<?, ?it/s]

...

                   all        822       1665       0.94      0.883      0.947       0.66
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 122, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



137 epochs completed in 4.831 hours.
Optimizer stripped from /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412070005-yolo11l/weights/last.pt, 51.2MB
Optimizer stripped from /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412070005-yolo11l/weights/best.pt, 51.2MB

Validating /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412070005-yolo11l/weights/best.pt...
Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11l summary (fused): 464 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:18<00:00,  1.39it/s]


                   all        822       1665       0.94       0.88      0.946      0.666
Speed: 0.3ms preprocess, 10.3ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412070005-yolo11l


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : extra_heron_8422
COMET INFO:     url                   : https://www.comet.com/victorcancer/content-drive-mydrive-uoc-delt-weights/bf8bf02aa32d4b23a18fa52322cc0647
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [275]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     lr/pg1 [275]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     lr/pg2 [275]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     metrics/mAP50(B) [276]     : (0.79925, 0.94972)
COMET INFO:     metrics/mAP50-95(B) [276]  : (0.43811, 0.66655)
COMET INFO:     metrics/precision(B) [276] : (0.7645, 0.9

In [ ]:
#@title Select YOLOv11 🚀 model {run: 'auto'}
YOLO_model = 'yolo11x' #@param ['yolo11n','yolo11s', 'yolo11m', 'yolo11l','yolo11x']

In [ ]:
dataset_name = 'osaconservation' + '-youtube'

# Cargamos el modelo
model = YOLO(YOLO_model)

date_string = datetime.now().strftime("%Y%m%d%H%M")
project_name = dataset_name + '-' + date_string + '-' + YOLO_model
project_folder = '/content/drive/MyDrive/UOC/DELT/weights'

# Train the model
results = model.train(data="DELTv11.yaml", epochs=300, freeze=11,patience = 15, project = project_folder, name = project_name)

100%|██████████| 109M/109M [00:01<00:00, 102MB/s]


New https://pypi.org/project/ultralytics/8.3.46 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11x.pt, data=DELTv11.yaml, epochs=300, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/UOC/DELT/weights, name=osaconservation-youtube-202412070456-yolo11x, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=11, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/victorcancer/content-drive-mydrive-uoc-delt-weights/7b0e4ebeb9b04d2097bafe73ad05512e

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


TensorBoard: Start with 'tensorboard --logdir /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412070456-yolo11x', view at http://localhost:6006/
Freezing layer 'model.0.conv.weight'
Freezing layer 'model.0.bn.weight'
Freezing layer 'model.0.bn.bias'
Freezing layer 'model.1.conv.weight'
Freezing layer 'model.1.bn.weight'
Freezing layer 'model.1.bn.bias'
Freezing layer 'model.2.cv1.conv.weight'
Freezing layer 'model.2.cv1.bn.weight'
Freezing layer 'model.2.cv1.bn.bias'
Freezing layer 'model.2.cv2.conv.weight'
Freezing layer 'model.2.cv2.bn.weight'
Freezing layer 'model.2.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv1.conv.weight'
Freezing layer 'model.2.m.0.cv1.bn.weight'
Freezing layer 'model.2.m.0.cv1.bn.bias'
Freezing layer 'model.2.m.0.cv2.conv.weight'
Freezing layer 'model.2.m.0.cv2.bn.weight'
Freezing layer 'model.2.m.0.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv3.conv.weight'
Freezing layer 'model.2.m.0.cv3.bn.weight'
Freezing layer 'model.2.m.0.cv3.bn.bias'
Freez

train: Scanning /content/datasets/DELT/train/labels.cache... 2601 images, 689 backgrounds, 2 corrupt: 100%|██████████| 3290/3290 [00:00<?, ?it/s]

...

                   all        822       1665      0.928       0.88      0.949      0.649
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 62, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



77 epochs completed in 3.942 hours.
Optimizer stripped from /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412070456-yolo11x/weights/last.pt, 114.4MB
Optimizer stripped from /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412070456-yolo11x/weights/best.pt, 114.4MB

Validating /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412070456-yolo11x/weights/best.pt...
Ultralytics 8.3.44 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11x summary (fused): 464 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:23<00:00,  1.09it/s]


                   all        822       1665      0.927      0.878      0.949      0.666
Speed: 0.2ms preprocess, 18.1ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /content/drive/MyDrive/UOC/DELT/weights/osaconservation-youtube-202412070456-yolo11x


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : required_caribou_5774
COMET INFO:     url                   : https://www.comet.com/victorcancer/content-drive-mydrive-uoc-delt-weights/7b0e4ebeb9b04d2097bafe73ad05512e
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [155]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     lr/pg1 [155]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     lr/pg2 [155]               : (0.00331715210355987, 0.009917925566343043)
COMET INFO:     metrics/mAP50(B) [156]     : (0.81259, 0.9492349666399171)
COMET INFO:     metrics/mAP50-95(B) [156]  : (0.49767, 0.6657674224644575)
COMET INFO:     metrics/precis

In [ ]:
from google.colab import runtime
runtime.unassign()